In [2]:
from tope import Tope
from tope.net import *
from tope.orth import *
from tests import normalize_polygon

import json, os, numpy as np
rng = np.random.default_rng()
with open("polys.json") as fd: polys = json.load(fd)
    
logger.remove()

In [ ]:
v_24cell = [
    [1, 0, 0, 0], [-1, 0, 0, 0],
    [0, 1, 0, 0], [0, -1, 0, 0],
    [0, 0, 1, 0], [0, 0, -1, 0], 
    [0, 0, 0, 1], [0, 0, 0, -1],
    [1, -1, 0, 0], [-1, 1, 0, 0], 
    [1, 0, -1, 0], [-1, 0, 1, 0], 
    [1, 0, 0, -1], [-1, 0, 0, 1], 
    [0, 1, -1, 0], [0, -1, 1, 0], 
    [0, 1, 0, -1], [0, -1, 0, 1],
    [1, 0, -1, -1], [-1, 0, 1, 1], 
    [0, 1, -1, -1], [0, -1, 1, 1],
    [1, 1, -1, -1], [-1, -1, 1, 1]
]

In [ ]:
EdgeList = list[np.ndarray] # list of 2xdim arrays

def get_cuts(N: Net) -> EdgeList:
    return np.concatenate([N.tope.metadata[2][i]["cuts"] for i in range(len(N.facets))])
        
        
def get_edges(N: Net) -> EdgeList: # apply to unfolded Net
    edges = []
    for i, vertices in N.facets.items():
        facet_template = N.tope.get_face(i) # has correct indices
        edges.extend((vertices[list(e)] for e in facet_template.faces[1]))
    return edges

import itertools

Net2d = None # new format of Net

def iter_edges(N: Net2d) -> Iterable[np.ndarray[2,2]]: # apply to unfolded 2d Net
    return N.iter_faces_as_vertices(dim=1)

FacetLabels = list[tuple[str, np.ndarray]]

def get_facet_labels(N: Net) -> FacetLabels:
    labels = []
    for i, vertices in N.facets.items():
        labels.append((N.tope.metadata[-1][i]["index"], vertices.mean(axis=0)))
    return labels

def iter_labels(N: Net2d, key: str) -> Iterable[str]:
    return zip(N.iter_meta(dim=2, key="index"), map(N.iter_cells(), lambda x: x.vertices.mean(axis=0)))

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection # container for line segments
from matplotlib.text import Text, Annotation
import matplotlib.cm as cm # colour mappings
import random

In [ ]:
def get_net_for_facet(P: Tope, i: int, keys=[]) -> Net:
    F = P.get_facet(i, metadata_keys=keys)
    G = get_facet_graph(F)
    T = G.get_spanning_tree()
    N = Net(F, T).unfold_with_metadata(meta_keys=keys).in_own_span(meta_keys=keys)
    return N

In [ ]:
MARGIN_FACTOR = 1.1

def save_nets(P: Tope, name: str, margin_factor: float = MARGIN_FACTOR, dpi: int = 300, force=False, fmt="png"):
    if os.path.exists(name):
        if force:
            if os.path.isdir(name):
                logger.warning("Directory already exists. Writing anyway.")
            else:
                logger.error("File already exists and is not a directory. Aborting")
                return
        else:
            logger.error("File already exists. Aborting.")
            return
    else:
        os.mkdir(name)
    
    nfacets = len(P.faces[-1])
    facet_net_list = []
    
    xlim = [0,0]
    ylim = [0,0]

    for i in range(nfacets):
        N = get_net_for_facet(P, i, keys=["cuts"])
        for face_v in N.facets.values():
            lower, upper = face_v.min(axis=0), face_v.max(axis=0)
            xlim[0] = min(xlim[0], lower[0])
            ylim[0] = min(ylim[0], lower[1])
            xlim[1] = max(xlim[1], upper[0])
            ylim[1] = max(ylim[1], upper[1])
        facet_net_list.append(N)
        
    xlim[0] *= margin_factor
    ylim[0] *= margin_factor
    xlim[1] *= margin_factor
    ylim[1] *= margin_factor
        
    for i in range(nfacets):
        fig, ax = plt.subplots()
        N = facet_net_list[i]
        lc = LineCollection(get_edges(N), colors="grey", linewidths=0.2)
        ax.add_collection(lc)
        cuts = LineCollection(get_cuts(N), colors="#c0c7e1", linewidths=0.1)
        ax.add_collection(cuts)
        for label, pos in get_facet_labels(N):
            ax.add_artist(Text(pos[0], pos[1], text=str(label), fontsize=2, ha="center", va="center"))
        
        # inset title in root face
        title_pos = N.facets[N.tree.root].mean(axis=0) + [0,-.4]
        ax.add_artist(Text(*title_pos, text=f"{name}\n(facet {i})", fontsize=2, ha="center", va="center"))

        ax.set_title(f"{name}-facet-{i}", y=1.0, pad=-14, fontsize=8)
        ax.set_xlim(*xlim)
        ax.set_ylim(*ylim)
        ax.set_aspect(1)
        
        fname = os.path.join(name, f"{i}.{fmt}")
        if force and os.path.exists(fname):
            logger.warning(f"Overwriting existing file {fname}.")
            try:
                os.unlink(fname)
            except OSError as err: # perhaps fname is a directory or we don't have the right permissions
                logger.error(str(err))
                return
        fig.savefig(fname, dpi=dpi)

In [ ]:
def get_hyperplanes(*planes, spacing=0.2, r=5):
    return [H for L in planes for H in [(L, i*L) for i in np.arange(-r, r, spacing)]]

hyperplanes = get_hyperplanes(
    *(rng.normal(size=4) for _ in range(4))
#    np.array([1,0,0,0]), np.array([0,1,0,0]), np.array([0,0,1,0]), np.array([0,0,0,1])
)

In [ ]:
P = Tope.from_vertices(v_24cell)
P.save_index() # now must do this before passing to sub-topes
P.cut_2faces_with_hyperplanes(hyperplanes) # and this
save_nets(P, "figs/24cell", force=True)

In [ ]:
P = Tope.from_vertices(polys["2070"])
P.save_index() # now must do this before passing to sub-topes
P.cut_2faces_with_hyperplanes(hyperplanes) # and this
save_nets(P, "figs/2070", force=True)

In [ ]:
# sanity check

P = Tope.from_vertices(v_24cell)
P.save_index() # now must do this before passing to sub-topes
P.cut_2faces_with_hyperplanes([(rng.normal(size=4), np.zeros(4)) for _ in range(1)]) # and this
#n = get_net_for_facet(P, 0)
#save_nets(P, "figs/24cell", force=True)
F = P.get_face(0,2)

origin = F.vertices.mean(axis=0)
v, basis = in_own_span(F.vertices-origin)
F1 = Tope(v, F.faces, F.metadata)
P.apply_to(lambda x : (x-origin)@basis.T, "cuts")

fig, ax = plt.subplots()
lc = LineCollection([F1.vertices[[0,1]], F1.vertices[[1,2]], F1.vertices[[2,0]], P.metadata[2][0]["cuts"][0]])
ax.add_collection(lc)
ax.autoscale()

In [ ]:
#save_nets(Tope.from_vertices(polys["2070"]), "figs/2070", force=True)

In [ ]:
for name, poly in polys.items():
    save_nets(Tope.from_vertices(poly), f"figs/{name}", force=True)

In [ ]:
with open("../polys1.json") as fd: data = json.load(fd)